In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def load_csv_auto_encoding(filepath):
    """
    자동 인코딩 감지를 시도하여 CSV 파일을 로드하는 함수입니다.
    일반적으로 사용되는 인코딩 몇 가지를 시도해보고, 그 중 하나로 파일을 성공적으로 열 수 있다면 해당 데이터프레임을 반환합니다.
    """
    encodings = ['utf-8', 'cp949', 'ISO-8859-1', 'cp1252']  # 시도할 인코딩 리스트
    for enc in encodings:
        try:
            df = pd.read_csv(filepath, encoding=enc)
            return df
        except UnicodeDecodeError:
            pass
    raise ValueError(f"All tried encodings failed. Check the file encoding: {filepath}")


def clean_data(df, method='median', fill_value=None, columns=None):
    """
    데이터 클리닝을 수행하는 함수입니다.
    결측치 처리 방법을 선택할 수 있습니다.
    method: 'mean', 'median', 'mode', 'constant' 중 하나를 선택합니다.
    fill_value: 'constant' 방법을 사용할 때 채울 값입니다.
    columns: 결측치를 처리할 특정 열(들)을 지정합니다. None일 경우 모든 열에 적용됩니다.
    """
    if columns is None:
        columns = df.columns

    if method == 'mean':
        # 평균으로 채우기
        df[columns].fillna(df[columns].mean(), inplace=True)
    elif method == 'median':
        # 중앙값으로 채우기
        df[columns].fillna(df[columns].median(), inplace=True)
    elif method == 'mode':
        # 최빈값으로 채우기 (범주형 데이터에 적합)
        for col in columns:
            df[col].fillna(df[col].mode()[0], inplace=True)
    elif method == 'constant':
        # 지정된 상수값으로 채우기
        if fill_value is not None:
            df[columns].fillna(fill_value, inplace=True)
        else:
            raise ValueError("fill_value must be provided for method='constant'")
    else:
        raise ValueError("Invalid method. Choose from 'mean', 'median', 'mode', 'constant'.")

    # 숫자가 아닌 데이터를 NaN으로 변환
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

# 사용예시
# CSV 파일 로드
#filepath = 'path_to_your_csv.csv'
#df = load_csv_auto_encoding(filepath)

# 모든 열에 대해 결측치를 중앙값으로 처리
#df = clean_data(df, method='median')

# 'column1'과 'column2' 열의 결측치를 최빈값으로 처리
#df = clean_data(df, method='mode', columns=['column1', 'column2'])

# 모든 열에 대해 결측치를 상수 0으로 처리
#df = clean_data(df, method='constant', fill_value=0)